# Разработка механизма ответов на вопросы за минуту

В этой тетради показано, как создать механизм ответов на вопросы с нуля  используя [Milvus](https://milvus.io/) и [Towhee](https://towhee.io/). Milvus - это самая совершенная векторная база данных с открытым исходным кодом, созданная для приложений искусственного интеллекта, и поддерживающая поиск ближайших соседей по десяткам миллионов записей, а Towhee - это платформа, которая предоставляет ETL для неструктурированных данных с использованием моделей машинного обучения SoTA.

Мы пройдемся по процедурам ответов на вопросы и оценим производительность. Более того, с помощью Towhee нам удалось упростить основную функциональность почти до 10 строк кода, так что вы можете начать взламывать свой собственный механизм ответов на вопросы.

## Подготовка

### Установите зависимости

Сначала нам нужно установить такие зависимости, как towhee, towhee.models и radio.

In [1]:
! python -m pip install -q towhee towhee.models gradio

### Подготовьте данные

Версия  [InsuranceQA Corpus](https://github.com/shuzi/insuranceQA)  В этой демонстрации используется часть корпуса вопросов и ответов по страхованию (1000 пар вопросов и ответов), которую каждый может скачать  [Github](https://github.com/towhee-io/examples/releases/download/data/question_answer.csv).

В этой демонстрации используется часть корпуса вопросов и ответов по страхованию (1000 пар вопросов и ответов), которую каждый может скачать на Github.

In [ ]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/question_answer.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  595k  100  595k    0     0   602k      0 --:--:-- --:--:-- --:--:--  602k


**question_answer.csv**: файл, содержащий вопрос и ответ.


Давайте кратко рассмотрим:

In [1]:
import pandas as pd

df = pd.read_csv('question_answer.csv')
df.head()

,id,question,answer
0,0,Is Disability Insurance Required By Law?,Not generally. There are five states that requ...
1,1,Can Creditors Take Life Insurance After ...,If the person who passed away was the one with...
2,2,Does Travelers Insurance Have Renters Ins...,One of the insurance carriers I represent is T...
3,3,Can I Drive A New Car Home Without Ins...,Most auto dealers will not let you drive the c...
4,4,Is The Cash Surrender Value Of Life Ins...,Cash surrender value comes only with Whole Lif...


Чтобы использовать набор данных для получения ответов, давайте сначала определим словарь:

- `id_answer`: словарь с id и соответствующим ответом

In [2]:
id_answer = df.set_index('id')['answer'].to_dict()

### Создание коллекции Milvus

Перед началом работы, пожалуйста, убедитесь, что у вас запущен  [Milvus service](https://milvus.io/docs/install_standalone-docker.md). В блокноте используется [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

In [5]:
! python -m pip install -q pymilvus==2.2.11

Затем определим функцию `create_milvus_collection` для создания коллекции в Milvus, которая использует [L2 distance metric](https://milvus.io/docs/metric.md#Euclidean-distance-L2) и [IVF_FLAT index](https://milvus.io/docs/index.md#IVF_FLAT).

In [2]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, MilvusClient

#connections.connect(host='172.17.0.1', port='19530')
client = connections.connect(
    uri="",
    token=""
)


In [5]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, MilvusClient



def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.VARCHAR, descrition='ids', max_length=500, is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection("quick_setup", 768)

## Механизм ответа на вопросы

В этом разделе мы покажем, как создать наш механизм ответов на вопросы с использованием Milvus и Towhee. Основная идея, лежащая в основе системы ответов на вопросы, заключается в том, чтобы использовать Towhee для создания вложений из набора данных вопросов и сравнения входного вопроса с вложениями, хранящимися в Milvus.

[Towhee](https://towhee.io/) это платформа машинного обучения, которая позволяет создавать конвейеры обработки данных, а также предоставляет предопределенные операторы для реализации операций вставки и запроса в Milvus.

<img src="./workflow.png" width = "60%" height = "60%" align=center />

### Загрузить вопрос, встраиваемый в Milvus

Сначала мы генерируем вложение из текста вопроса с помощью оператора[dpr](https://towhee.io/text-embedding/dpr) и вставляем вложение в Milvus. Towhee предоставляет [method-chaining style API](https://towhee.readthedocs.io/en/main/index.html) пользователи могли создавать конвейер обработки данных с операторами.

In [5]:
%%time
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection

insert_pipe = (
    pipe.input('id', 'question', 'answer')
        .map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec'), 'insert_status', ops.ann_insert.milvus_client(uri="", token='you token', collection_name='question_answer'))
        .output()
)

import csv
with open('question_answer.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        insert_pipe(*row)

2024-05-14 20:17:24,524 - 140503368794176 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-05-14 20:17:24,760 - 140503368794176 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/config.json HTTP/1.1" 200 0
2024-05-14 20:17:24,937 - 140503368794176 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/config.json HTTP/1.1" 200 0
2024-05-14 20:17:24,949 - 140503368794176 - connectionpool.py-connectionpool:1055 - DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2024-05-14 20:17:25,368 - 140503368794176 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/model.safetensors HTTP/1.1" 404 0
2024-05-14 20:17:27

CPU times: user 6min 35s, sys: 11.9 s, total: 6min 47s
Wall time: 8min 22s


In [6]:
print('Total number of inserted data is {}.'.format(collection.num_entities))

Total number of inserted data is 1000.


#### Объяснение конвейера обработки данных

Вот подробное объяснение для каждой строки кода:

`pipe.input('id', 'question', 'answer')`: Получаем три входных сигнала, а именно идентификатор вопроса, текст запроса и ответ на вопрос;

`map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))`: Use the `acebook/dpr-ctx_encoder-single-nq-base` модель для генерации вектора встраивания вопроса с помощью [dpr operator](https://towhee.io/text-embedding/dpr) in towhee hub;

`map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))`: нормализация векторов встраивания;

`map(('id', 'vec'), 'insert_status', ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='question_answer'))`: вставить вектор встраивания вопроса в Milvus;

### Задайте вопрос Милвусу и Тови

Теперь, когда набор данных для встраивания вопросов был вставлен в Milvus, мы можем задать вопрос с помощью Milvus и Towhee. И снова мы используем Towhee для загрузки входного вопроса, вычисления встраивания и использования его в качестве запроса в Milvus. Поскольку Milvus выводит только идентификаторы и значения расстояния, мы предоставляем словарь `id_answers` , чтобы получать ответы на основе идентификаторов и отображения.

In [32]:
%%time
collection.load()
ans_pipe = (
    pipe.input('question')
        .map('question', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client( uri= '',token= '', collection_name='testuser', limit=1))
        .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
        .output('question', 'answer')
)


ans = ans_pipe('Is  Disability  Insurance  Required  By  Law?')
ans = DataCollection(ans)
ans.show()

2024-04-01 20:39:42,857 - 140540341481536 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-04-01 20:39:43,101 - 140540341481536 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/config.json HTTP/1.1" 200 0
2024-04-01 20:39:43,280 - 140540341481536 - connectionpool.py-connectionpool:549 - DEBUG: https://huggingface.co:443 "HEAD /facebook/dpr-ctx_encoder-single-nq-base/resolve/main/config.json HTTP/1.1" 200 0
2024-04-01 20:39:47,723 - 140535374272192 - node.py-node:167 - INFO: Begin to run Node-_input
2024-04-01 20:39:47,729 - 140538862892736 - node.py-node:167 - INFO: Begin to run Node-text-embedding/dpr-0
2024-04-01 20:39:47,741 - 140535398397632 - node.py-node:167 - INFO: Begin to run Node-lambda-1
2024-04-01 20:39:47,745 - 140538904831680 - node.py-node:167 - INFO: Begin to

question,answer
Is Disability Insurance Required By Law?,Not generally. There are five states that require most all employers carry short term disability insurance on their employees. T...


CPU times: user 1.9 s, sys: 1.64 s, total: 3.54 s
Wall time: 11.7 s


Тогда мы сможем получить ответ на вопрос: "Требуется ли Страхование по Инвалидности По Закону?".

In [33]:
ans[0]['answer']

['Not generally. There are five states that require most all employers carry short term disability insurance on their employees. These states are: California, Hawaii, New Jersey, New York, and Rhode Island. Besides this mandatory short term disability law, there is no other legislative imperative for someone to purchase or be covered by disability insurance.']

## Презентация релиза

Мы отлично поработали над основными функциональными возможностями нашего механизма ответов на вопросы. Теперь пришло время создать демонстрационную версию с интерфейсом.[Gradio](https://gradio.app/) - отличный инструмент для создания демонстрационных материалов. С Gradio нам просто нужно завершить процесс обработки данных с помощью функции `chat`:

In [34]:
import towhee
def chat(message, history):
    history = history or []
    ans_pipe = (
        pipe.input('question')
            .map('question', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
            .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
            .map('vec', 'res', ops.ann_search.milvus_client(uri= '',token= '', collection_name='testuser2', limit=1))
            .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
            .output('question', 'answer')
    )

    response = ans_pipe(message).get()[1][0]
    history.append((message, response))
    return history, history

In [ ]:
import gradio

collection.load()
chatbot = gradio.Chatbot(color_map=("green", "gray"))
interface = gradio.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
interface.launch(inline=True, share=True)